In [ ]:
pip install solar-wavelength-calibration

In [ ]:
from solar_wavelength_calibration import WavelengthCalibrationFitter, WavelengthCalibrationParameters
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from astropy.wcs import WCS

## Basic Usage

In this section, you'll explore how to use the `solar-wavelength-calibration` package. Try to fill in the missing code and answer questions about each step.

Define the spectrum to be fit and compute the expected wavelength vector:

In [ ]:
# TODO: Load the characteristic spectrum and the initial wavelength vector
input_spectrum = np.load()

input_wavelength_vector = np.load()

Set up the model parameters (if you are implementing this into your pipeline, these values should be specific to your instrument):

In [ ]:
input_parameters = WavelengthCalibrationParameters(
        crval=350.159 * u.nm,
        dispersion=4.042 * u.nm/u.pix,
        incident_light_angle=57.006 * u.deg,
        resolving_power=42500,
        opacity_factor=5.0,
        straylight_fraction=0.2,
        grating_constant=31600.0 * 1/u.m,
        doppler_velocity=-0.428 * u.km/u.s,
        order=52,
    )


Take a look at the package documentation (https://bitbucket.org/dkistdc/solar-wavelength-calibration/src/main/?search_id=3c0636db-cb76-4242-ae9b-bc580f4e956c) and initialize the fitter and perform the fit:

Initialize the wavelength calibration fitter:

In [ ]:
# TODO: Complete the following code
fitter = WavelengthCalibrationFitter(
        input_parameters=input_parameters,
    )

Perform the wavelength calibration fit:

In [ ]:
# TODO: Complete the following code
fit_result = fitter(
        input_wavelength_vector=input_wavelength_vector,
        input_spectrum=input_spectrum,
    )


Check out the results!

In [ ]:
corrected_header = fit_result.wavelength_parameters.to_header(axis_num=1)
# If you want, there is an arg to add alternate keys (FITS 'A' keys). Check out the documentation for how to do this!

corrected_header

In [ ]:
corrected_wcs = WCS(corrected_header)

number_of_wave_pix = np.size(input_spectrum)

fitted_wave = (corrected_wcs.spectral.pixel_to_world(np.arange(number_of_wave_pix))).to_value(u.nm)

aligned_solar_transmission = fit_result.minimizer_result.residual + input_spectrum

fig, ax = plt.subplots(3, 1, figsize=(7, 6), height_ratios=[1, 0.4, 0.4], sharex=True)
ax[0].plot(input_wavelength_vector, input_spectrum, label="Input Spectrum")
ax[0].plot(
    fitted_wave,
    aligned_solar_transmission,
    label="Doppler-Shifted Solar Atlas",
    linestyle="-",
    color="k",
    alpha=0.3,
)
ax[0].plot(fitted_wave, input_spectrum, label="Corrected Spectrum")
ax[0].legend()
ax[0].set_ylabel("Intensity")
residuals = fit_result.minimizer_result.residual
ax[1].plot(fitted_wave, residuals, label="Residuals")
ax[1].legend()
fig.suptitle("Corrected Spectrum vs Input Spectrum")
fig.tight_layout()
plt.show()

## Advanced Usage

Using your own reference atlas

Smart CRVAL initial parameter estimation

Setting bounds for the model

Setting weights for the model

Changing the optimization method

Freezing parameters